In [ ]:
!pip install --upgrade pip setuptools wheel

In [ ]:
#Step 1: Import the Necessary Libraries
!pip install chromaDB
!pip install pypdf
!pip install openai
!pip install -U langchain-openai
!pip install -U langchain-chroma
!pip install gradio

In [ ]:
#Import essential tools and set up OpenAI's API environment.
import os

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain.document_loaders import PyPDFLoader

from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate
)

from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI


import chromadb
from chromadb.utils import embedding_functions
from openai import OpenAI

chroma_client = chromadb.Client()
os.environ['OPENAI_API_KEY'] = open_api_key


In [ ]:
#Load Nestle's HR policy using PyPDFLoader and split it for easy processing
pdf_loader = PyPDFLoader("nestle_hr_policy_pdf_2012.pdf")
pdf_pages = pdf_loader.load_and_split()
print(f"Loaded  {len(pdf_pages)} pages from Nestle's HR Policy.")
#print(pdf_pages[0].page_content[:100])

In [ ]:
#Step-3: Create vector representations for text chunks using Chroma dB and OpenAI's embeddings.

In [ ]:
#Chunk the document

#from langchain.document_loaders import PyPDFLoader
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200,length_function=len)
documents = text_splitter.split_documents(pdf_pages)
print(f"Split into {len(documents)} text chuncks for processing.")


In [ ]:
#Create Embeddings using chromaDB and Open AI Embeddings
# Initialize your embedding model
openai_embed = OpenAIEmbeddings()


In [ ]:
#!pip install -U langchain-chroma

In [ ]:
persist_directory = "nestle_hr_policy_db"
vector_str = Chroma.from_documents(
    documents=documents,
    embedding=openai_embed,
    persist_directory=persist_directory    
)
#vector_str.persist()
print(f"Vector Database created and saved to {persist_directory}")


In [ ]:
# Create a prompt template to guide the chatbot
template = """You are an HR assistant for Nestle, helping employees understand company HR policies.
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Keep your answers professional, helpful, and concise.

Context:
{context}

Question: {question}
Answer:"""

PROMPT = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

In [ ]:
 #Build a question-answering system using the GPT-3.5 Turbo model to retrieve answers from text chunks.

In [ ]:
from langchain.chains import RetrievalQA
retriever = vector_str.as_retriever(search_kwargs={"k": 1})
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)


In [ ]:
def query_policy(query):
    result = qa_chain({"query":query})
    print(f"Question:{query}")
    print(f"Answer: {result['result']}")
    print("\nSources:")
    for i, doc in enumerate(result["source_documents"]):
        print(f"Source {i+1} (Page {doc.metadata.get('page', 'Unknown')}):")
        print(f"{doc.page_content[:1000]}...\n")
 

In [ ]:
query_1="what is the Nestle's policy on Total Rewards?"
query_policy(query_1)

In [ ]:
query_2="what is the Nestle's policy on training and learning?"
query_policy(query_2)

In [ ]:
query_3="what are the Employment and Working Conditions at Nestle?"
query_policy(query_3)

In [ ]:
query_4="what is meant by Shared Responsibilty at Nestle?"
query_policy(query_4)

In [ ]:
#Step-6 : Use Gradio to build a user-friendly chatbot interface, enabling interaction and information retrieval.

In [ ]:
#Create a function for Gradio Interface

def process_query(message, history):
    result=qa_chain({"query":message})
    answer=result["result"]
    return answer

    

In [ ]:
import gradio as gr

with gr.Blocks(css="footer {visibility: hidden}") as demo:
    gr.Markdown("# Nestle HR Policy Assistant")
    gr.Markdown("""Ask any questions about Nestle's HR policies and get instant answers.
                This assistant has been trained on the official Nestle HR policy documents.""")
      
    chatbot = gr.Chatbot(
        [],
        elem_id="chatbot",
        height=500,
        type="tuples"  
    )
    
    msg = gr.Textbox(
        placeholder="Ask about Nestle's HR policies",
        container=False,
        scale=7
    )
    
    with gr.Row():
        submit = gr.Button("Submit", variant="primary", scale=1)
        clear = gr.Button("Clear", variant="secondary", scale=1)
    
    gr.Examples(
        examples=[
            "what is the Nestle's policy on Total Rewards?",
            "what is the Nestle's policy on training and learning?",
            "what are the Employment and Working Conditions at Nestle?",
            "what is meant by Shared Responsibilty at Nestle?",
            "On What basis, the performance management is developed at Nestle?"
        ],
        inputs=msg
    )
    # Set up event handlers with correct chat history handling
    def user_input(user_message, history):
        # Return the updated history and clear the textbox
        return "", history + [[user_message, None]]
    
    def bot_response(history):
        # Get the last user message
        user_message = history[-1][0]
        # Get bot response
        bot_message = process_query(user_message, history[:-1])
        # Update the last message with the bot's response
        history[-1][1] = bot_message
        return history
    
    # Chain the events properly
    msg.submit(user_input, [msg, chatbot], [msg, chatbot]).then(
        bot_response, chatbot, chatbot
    )
    
    submit.click(user_input, [msg, chatbot], [msg, chatbot]).then(
        bot_response, chatbot, chatbot
    )
    
    clear.click(lambda: [], None, [chatbot], queue=False)

In [ ]:
demo.launch(share=True, inbrowser=True)